# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [427]:
import sqlite3

In [428]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [429]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
CLIENT:
  0: NB(INT)
  1: NOM(VARCHAR)
  2: PRENOM(VARCHAR)
  3: TYPE(VARCHAR)
CONSUMPTION:
  0: NV(INT)
  1: NB(INT)
  2: DATES(VARCHAR)
  3: LIEU(VARCHAR)
  4: QTE(INT)
LOCALISATION:
  0: LIEU(VARCHAR)
  1: REGION(VARCHAR¨)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCER:
  0: NP(INT)
  1: NOM(VARCHAR)
  2: PRENOM(VARCHAR)
  3: REGION(VARCHAR)
PRODUCTION:
  0: NV(INT)
  1: NP(INT)
  2: QTE(INT)
RBA:
  0: nb(INT)
  1: nom(VARCHAR)
  2: prenom(VARCHAR)
  3: TYPE(VARCHAR)
RBB:
  0: nb(INT)
  1: nom(VARCHAR)
  2: prenom(VARCHAR)
WINE:
  0: NV(INT)
  1: CRU(VARCHAR)
  2: DEGRE(VARCGAR)
  3: MILL(INT)
WINECONSUMED:
  0: NV(INT)
  1: CRU(VARCHAR)
  2: DEGRE(VARCHAR)
  3: MILL

Again, we will use **`%%sql`** magic for our queries

In [430]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [431]:
%%sql DROP TABLE IF EXISTS PRODUCER;
CREATE TABLE PRODUCER(NP INT, NOM VARCHAR, PRENOM VARCHAR, REGION VARCHAR);

INSERT INTO PRODUCER (NP, NOM, PRENOM, REGION)
SELECT NP, NOM, PRENOM, REGION
FROM Master1;

DROP TABLE IF EXISTS WINE;
CREATE TABLE WINE(NV INT, CRU VARCHAR, DEGRE VARCGAR, MILL INT);

INSERT INTO WINE (NV, CRU, DEGRE, MILL)
SELECT NV, CRU, DEGRE, MILL
FROM Master1;

DROP TABLE IF EXISTS PRODUCTION;
CREATE TABLE PRODUCTION(NV INT, NP INT, QTE INT);

INSERT INTO PRODUCTION (NV, NP, QTE)
SELECT NV, NP, QTE
FROM Master1;

DROP TABLE IF EXISTS CLIENT;
CREATE TABLE CLIENT(NB INT, NOM VARCHAR, PRENOM VARCHAR, TYPE VARCHAR);

INSERT INTO CLIENT (NB, NOM, PRENOM, TYPE)
SELECT distinct NB, NOM, PRENOM, TYPE
FROM Master2
where nb is not null
;

DROP TABLE IF EXISTS CONSUMPTION;
CREATE TABLE CONSUMPTION(NV INT, NB INT, DATES VARCHAR, LIEU VARCHAR, QTE INT);

INSERT INTO CONSUMPTION (NV, NB, DATES, LIEU, QTE)
SELECT NV, NB, DATES, LIEU, QTE
FROM Master2;

DROP TABLE IF EXISTS LOCALISATION;
CREATE TABLE LOCALISATION(LIEU VARCHAR, REGION VARCHAR¨);

INSERT INTO LOCALISATION (LIEU, REGION)
SELECT LIEU, REGION
FROM Master2;

 * sqlite:///wine.db
Done.
Done.
210 rows affected.
Done.
Done.
210 rows affected.
Done.
Done.
210 rows affected.
Done.
Done.
100 rows affected.
Done.
Done.
185 rows affected.
Done.
Done.
185 rows affected.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [432]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;

CREATE view bons_buveurs as
select distinct nb, nom, prenom 
from client
where type in ('gros', 'moyen');

 * sqlite:///wine.db
Done.
Done.


[]

In [433]:
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
10,Andersen,Yann
12,Bataille,Georges
13,Barthes,Roland
14,Bory,Jean Louis


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [434]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;

CREATE view buveurs_asec as
select distinct client.nb, nom, prenom 
from client join consumption on client.nb = consumption.nb 
where qte is null;

 * sqlite:///wine.db
Done.
Done.


[]

In [435]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
11,Breton,Andre
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
21,Corbiere,Tristan
23,Corneille,Pierre
25,Char,Rene
27,Dumas,Alexandre
29,Fournier,Alain


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [436]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;

CREATE view buveurs_achats as
select distinct client.nb, nom, prenom 
from client 
    join consumption on client.nb = consumption.nb 
where client.nb not in (
    select distinct client.nb
    from client join consumption on client.nb = consumption.nb 
    where qte is null)

 * sqlite:///wine.db
Done.
Done.


[]

In [437]:
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
9,Ajar,Emile
10,Andersen,Yann


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [438]:
%%sql
DROP VIEW IF EXISTS q83pl;

CREATE view q83pl as
select distinct lieu, cru, qte
from consumption 
    join wine on wine.nv = consumption.nv
where dates between "1983-01-01" and "1983-12-31"
group by lieu, cru

 * sqlite:///wine.db
Done.
Done.


[]

In [439]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,130
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

Answer : Yes, as a view can be used as a table even though it is not an object in itself

In [440]:
%%sql
DROP VIEW IF EXISTS q83pl_asc;

CREATE view q83pl_asc as
select distinct lieu, cru, qte
from consumption 
    join wine on wine.nv = consumption.nv
where dates between "1983-01-01" and "1983-12-31"
group by lieu, cru
order by qte asc;

SELECT * FROM q83pl order by QTE;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,CRU,QTE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,130


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [441]:
%%sql

Drop table if exists RBB;
Create table RBB(nb INT, nom VARCHAR, prenom VARCHAR);
Insert into RBB(nb, nom, prenom)
select nb, nom, prenom
from bons_buveurs;

 * sqlite:///wine.db
Done.
Done.
56 rows affected.


[]

In [442]:
%sql SELECT * FROM RBB order by nb;

 * sqlite:///wine.db
Done.


nb,nom,prenom
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
10,Andersen,Yann
12,Bataille,Georges
13,Barthes,Roland
14,Bory,Jean Louis


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [443]:
%%sql
select client.nb, nom, prenom, type, sum(consumption.qte)
from client join consumption
    on client.nb = consumption.nb
group by consumption.nb
having sum(consumption.qte) > 100 and type <> 'gros'

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,sum(consumption.qte)
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances :

In [444]:
%%sql 
select count(nb)
from client
where type ='gros'

 * sqlite:///wine.db
Done.


count(nb)
34


In [445]:
%%sql
UPDATE CLIENT
SET TYPE = 'gros' 
WHERE nb in 
    (select client.nb
    from client join consumption
        on client.nb = consumption.nb
        group by client.nb
    having sum(consumption.qte) > 100 and TYPE <> 'gros');

 * sqlite:///wine.db
4 rows affected.


[]

In [446]:
%%sql 
select count(nb)
from client
where type = 'gros'

 * sqlite:///wine.db
Done.


count(nb)
38


4 new customers have been added to the table Client with type 'Gros'.

In [458]:
%%sql
select client.nb, nom, prenom, type, sum(consumption.qte) as total
from client join consumption
    on client.nb = consumption.nb
group by client.nb
having total > 100 and type <> 'gros'

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,total


Table is now empty.

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [459]:
%sql select * from rbb

 * sqlite:///wine.db
Done.


nb,nom,prenom
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
27,Dumas,Alexandre
32,Eluard,Paul
35,Fromentin,Eugene
39,Montesquieu,None
42,Goethe,Johann Wolfgang von
43,Musset,Alfred de


In [449]:
%sql select * from bons_buveurs;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
27,Dumas,Alexandre
32,Eluard,Paul
35,Fromentin,Eugene
39,Montesquieu,None
42,Goethe,Johann Wolfgang von
43,Musset,Alfred de


In [450]:
%sql select count(nb) from rbb

 * sqlite:///wine.db
Done.


count(nb)
56


In [451]:
%sql select count(NB) from bons_buveurs

 * sqlite:///wine.db
Done.


count(NB)
59


Increase of 3 in the size of bons_buveurs is logic since we added 3 new 'petit' buveurs to the table of medium or big ones.

Answer: bons_buveurs is now longer than RBB since it contains the data regarding medium and big consumers and we just added new big consumers. Views are not tables, so when underlying datas are updated, they are updated automatically.

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [452]:
%%sql

Drop table if exists RBA;

Create table RBA(nb INT, nom VARCHAR, prenom VARCHAR);

Insert into RBA(nb, nom, prenom)
select nb, nom, prenom
from buveurs_asec;

ALTER TABLE rba ADD COLUMN TYPE VARCHAR;

UPDATE rba
SET TYPE = (SELECT TYPE FROM client WHERE RBA.nb = client.nb);

 * sqlite:///wine.db
Done.
Done.
57 rows affected.
Done.
57 rows affected.


[]

In [453]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


nb,nom,prenom,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [454]:
%%sql
insert into rba ('nb', 'nom', 'prenom', 'type')
values (101, 'FABIEN', 'MAEL', 'moyen')

 * sqlite:///wine.db
1 rows affected.


[]

In [455]:
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


nb,nom,prenom,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [456]:
%sql SELECT count(nb) FROM RBA

 * sqlite:///wine.db
Done.


count(nb)
58


In [457]:
%sql SELECT count(nb) FROM buveurs_asec

 * sqlite:///wine.db
Done.


count(nb)
57


Answer: In buveurs_asec, the datas have not been updated since the source of the data has not been modified. However, in RBA, the table has been updated.

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

<img src="Schema2.png">

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.